In [ ]:
!pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.5 MB/s eta 0:00:00


In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from skmultilearn.model_selection import IterativeStratification, iterative_train_test_split

In [ ]:
!!wget -nc -O CRA_train_1200.7z "https://docs.google.com/uc?export=download&id=13oPXaBZmDL1kJFcK-KR4_K-VAKSX7Q3d"
!7z x CRA_train_1200.7z  -aos

In [ ]:
maxSequenceLength = 256

In [ ]:
df = pd.read_csv('CRA_train_1200.csv')

In [ ]:
df

,pr_txt,Категория,Уровень рейтинга
0,Повышение кредитного рейтинга Акционерного об...,A,A
1,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB
2,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A
3,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA
4,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB
...,...,...,...
1195,«Эксперт РА» подтвердил рейтинг компании «Татн...,AAA,AAA
1196,«Эксперт РА» понизил рейтинг компании «Инвестг...,BB,BB
1197,АКРА подтвердило ПАО «КАМАЗ» кредитный рейтинг...,A,A+
1198,АКРА подтвердило кредитный рейтинг ФГУП «Почта...,AAA,AAA


In [ ]:
txt_lengths = [len(t.split()) for t in df['pr_txt']]
pd.Series(txt_lengths).hist(bins=25)

In [ ]:
pd.Series(txt_lengths).describe()

count    1200.000000
mean     1232.509167
std       323.565958
min       465.000000
25%      1014.000000
50%      1230.000000
75%      1454.000000
max      2827.000000
dtype: float64

In [ ]:
for index, row in df.iterrows():
    split_text = [x for x in row['pr_txt'].split() if x]
    df.loc[index, 'pr_txt'] = ' '.join(split_text[0: maxSequenceLength])
    for i in range(maxSequenceLength, len(split_text), maxSequenceLength):
        df.loc[len(df)] = {
            'pr_txt': ' '.join(split_text[i: i + maxSequenceLength]),
            'Уровень рейтинга': row['Уровень рейтинга'],
            'Категория': row['Категория']
        }

In [ ]:
df['Категория'].unique()

array(['A', 'BB', 'AAA', 'BBB', 'AA', 'B', 'C'], dtype=object)

In [ ]:
df['Уровень рейтинга'].unique()

array(['A', 'BB', 'AAA', 'BBB', 'AA+', 'BB+', 'BB-', 'A-', 'A+', 'B',
       'AA-', 'BBB+', 'BBB-', 'B-', 'AA', 'B+', 'C'], dtype=object)

In [ ]:
categories = {}
for key, value in enumerate(df['Категория'].unique()):
    categories[value] = key

# Запишем в новую колонку числовое обозначение категории
df['label'] = df['Категория'].map(categories)

total_categories = len(df['label'].unique()) + 1
print('Всего категорий: {}'.format(total_categories))

Всего категорий: 8


In [ ]:
sentence = df['pr_txt'].tolist()
label = df['label'].tolist()
num_labels = len(np.unique(label))

In [ ]:
def one_hot_encode(y, n_classes=None):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : y: List of sample Labels
    : return: Numpy array of one-hot encoded labels
     """
    if n_classes is None:
        n_classes = len(np.unique(y))
    return np.eye(n_classes)[y]


def iterative_train_test_split(X, y, test_size):
    """Iteratively stratified train/test split

    Parameters
    ----------
    test_size : float, [0,1]
        the proportion of the dataset to include in the test split, the rest will be put in the train set

    Returns
    -------
    X_train, y_train, X_test, y_test
        stratified division into train/test split
    """

    stratifier = IterativeStratification(n_splits=2, order=2, sample_distribution_per_fold=[test_size, 1.0-test_size])
    tmp_y = one_hot_encode(y)
    train_indexes, test_indexes = next(stratifier.split(X, tmp_y))

    X_train, y_train = np.array(X)[train_indexes.astype(int)], np.array(y)[train_indexes.astype(int)]
    X_test, y_test = np.array(X)[test_indexes.astype(int)], np.array(y)[test_indexes.astype(int)]

    return X_train, X_test, y_train, y_test

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(descriptions, categories_code, test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = iterative_train_test_split(sentence, label, test_size=0.15)
# Приведение массива меток categories_code к массиву numpy.
all_y = np.array(label)
#Вызов метода class_weight.compute_class_weight() для назнаячения весов меткам данных.
class_weights_ist = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(all_y),
    y=all_y
)

In [ ]:
print('\n'.join(f"{x}\t{y}" for x, y in zip(categories.keys(), class_weights_ist)))

A	0.44025157232704404
BB	1.2247644683714671
AAA	1.0769230769230769
BBB	0.6637490882567469
AA	0.8512628624883068
B	3.823529411764706
C	24.594594594594593


In [ ]:
print('\n'.join(f"{x}\t{y}" for x, y in zip(categories.keys(), class_weights_ist)))

A	0.636172975132328
BB	1.3981562774363476
AAA	0.4434389140271493
BBB	0.8217234262125903
AA+	0.9886698742821667
BB+	1.193330835518921
BB-	2.3273657289002556
A-	0.47551507912809793
A+	0.5430520034100597
B	5.764705882352941
AA-	1.2448700410396716
BBB+	0.6427202098678236
BBB-	1.128632175761871
B-	5.764705882352941
AA	0.9632541962800545
B+	3.469498910675381
C	10.127186009538951


In [ ]:
!pip install transformers[torch] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 17.6 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    TrainingArguments, Trainer
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2', num_labels=num_labels).to(device)
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length=maxSequenceLength,
    padding='max_length',
    truncation=True
)
tokens_test = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length=maxSequenceLength,
    padding='max_length',
    truncation=True
)

In [ ]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Data(tokens_train, y_train)
test_dataset = Data(tokens_test, y_test)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")

    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        #'macro f1': macro_f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
class_weights_ist = torch.tensor(class_weights_ist, dtype=torch.float).to(device)
loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_ist)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir="./results", #Выходной каталог
    num_train_epochs=30, #Кол-во эпох для обучения
    per_device_train_batch_size = 64, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 64, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir="./logs", #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    metric_for_best_model="f1",
    save_total_limit = 1,
    seed=21)

In [ ]:
trainer = CustomTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.177200,0.813902,0.786611,0.760547,0.740865,0.799665
2,0.197500,0.830974,0.788703,0.763566,0.750860,0.797310
3,0.178100,0.858038,0.789749,0.745036,0.735860,0.790511
4,0.177400,0.815550,0.789749,0.755225,0.737260,0.799431
5,0.170700,0.937442,0.785565,0.745336,0.744613,0.776626
6,0.183000,0.807775,0.790795,0.776384,0.761374,0.801630
7,0.173600,0.834349,0.783473,0.739262,0.725407,0.792939
8,0.168700,0.849884,0.794979,0.745792,0.731707,0.797885
9,0.166900,0.880248,0.787657,0.743457,0.734142,0.789063
10,0.164700,0.833435,0.791841,0.755998,0.737962,0.803927


TrainOutput(global_step=1700, training_loss=0.17106926244847914, metrics={'train_runtime': 165.7926, 'train_samples_per_second': 653.105, 'train_steps_per_second': 10.254, 'total_flos': 399499853475840.0, 'train_loss': 0.17106926244847914, 'epoch': 20.0})

In [ ]:
model_path = "rubert-CB-cat-7764"

In [ ]:
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('rubert-CB-cat-7764\\tokenizer_config.json',
 'rubert-CB-cat-7764\\special_tokens_map.json',
 'rubert-CB-cat-7764\\vocab.txt',
 'rubert-CB-cat-7764\\added_tokens.json')

In [ ]:
categories_str = dict((str(k), v) for k, v in categories.items())

with open(f'{model_path}/categories.json', 'w') as f:
    json.dump(categories_str, f)

In [ ]:
inv_categories = {v: k for k, v in categories.items()}

In [ ]:
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()

In [ ]:
inv_categories

{0: 'A', 1: 'BB', 2: 'AAA', 3: 'BBB', 4: 'AA', 5: 'B', 6: 'C'}

In [ ]:
model.config.id2label = inv_categories
model.config.id2label

{0: 'A', 1: 'BB', 2: 'AAA', 3: 'BBB', 4: 'AA', 5: 'B', 6: 'C'}

In [ ]:
model.config.label2id = categories

In [ ]:
print(classification_report(y_test, pred, target_names=categories.keys(), digits=4))

              precision    recall  f1-score   support

           A     0.8155    0.8129    0.8142       310
          BB     0.7207    0.7207    0.7207       111
         AAA     0.9130    0.8268    0.8678       127
         BBB     0.7395    0.7718    0.7553       206
          AA     0.8075    0.8125    0.8100       160
           B     0.6667    0.6667    0.6667        36
           C     0.6667    1.0000    0.8000         6

    accuracy                         0.7908       956
   macro avg     0.7614    0.8016    0.7764       956
weighted avg     0.7932    0.7908    0.7914       956



In [ ]:
print(classification_report(y_test, pred, target_names=categories.keys(), digits=4))

              precision    recall  f1-score   support

           A     0.8613    0.8613    0.8613       310
          BB     0.7570    0.7297    0.7431       111
         AAA     0.9273    0.8031    0.8608       127
         BBB     0.7661    0.8107    0.7877       206
          AA     0.8023    0.8875    0.8427       160
           B     0.7500    0.5000    0.6000        36
           C     0.6000    1.0000    0.7500         6

    accuracy                         0.8190       956
   macro avg     0.7806    0.7989    0.7779       956
weighted avg     0.8217    0.8190    0.8180       956



In [ ]:
!7z a {model_path} {'./' + model_path}

In [ ]:
root_path = '/content/drive/MyDrive/Colab Notebooks/ЦБ оценка рейтинга пресс-релизов'

In [ ]:
from shutil import copyfile, copytree

In [ ]:
copyfile(model_path + '.7z', f"{root_path}/{model_path}.7z")

'/content/drive/MyDrive/Colab Notebooks/ЦБ оценка рейтинга пресс-релизов/rubert-CB-6536.7z'

In [ ]:
copyfile( f"{root_path}/{model_path}.7z", model_path + '.7z')

'rubert-CB-6536.7z'

In [ ]:
!7z x {model_path + '.7z'}  -aos

In [ ]:
model_path = f"{root_path}/{model_path}"

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = BertTokenizer.from_pretrained(model_path)

In [ ]:
copytree(model_path, f"{root_path}/{model_path}")

'/content/drive/MyDrive/Colab Notebooks/ЦБ оценка рейтинга пресс-релизов/rubert-CB-cat-7779'

In [ ]:
def explain(model, sample):
    """ Computes the effect each word had on model predictions """
    sample_col_tokens = sample.split()
    data_drop_one_token = [sample] * (len(sample_col_tokens) + 1)

    for drop_i in range(len(sample_col_tokens)):
        data_drop_one_token[drop_i] = ' '.join(tokenizer.unk_token if i == drop_i else tok
                                               for i, tok in enumerate(sample_col_tokens))
    x_tmp = tokenizer(
        data_drop_one_token,
        return_tensors="pt",
        max_length=maxSequenceLength,
        padding='max_length',
        truncation=True
    ).to(device)
    with torch.no_grad():
        *predictions_drop_one_token, baseline_pred = model(**x_tmp).logits.detach().cpu()
    predicted_class_id = baseline_pred.argmax().item()
    predictions_drop_one_token = torch.stack(predictions_drop_one_token)
    diffs = baseline_pred[predicted_class_id] - predictions_drop_one_token[:, predicted_class_id]
    return list(zip(sample_col_tokens, diffs))

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import HTML, display_html


def draw_html(tokens_and_weights, cmap=plt.get_cmap("bwr"), display=True,
              token_template="""<span style="background-color: {color_hex}">{token}</span>""",
              font_style="font-size:14px;"
             ):

    def get_color_hex(weight):
        rgba = cmap(1. / (1 + np.exp(float(weight))), bytes=True)
        return '#%02X%02X%02X' % rgba[:3]

    tokens_html = [
        token_template.format(token=token, color_hex=get_color_hex(weight))
        for token, weight in tokens_and_weights
    ]

    raw_html = """<p style="{}">{}</p>""".format(font_style, ' '.join(tokens_html))
    if display:
        display_html(HTML(raw_html))

    return raw_html

In [ ]:
tokens_and_weights = list(zip(sample_col_tokens, diffs))

'«Эксперт РА» присвоил кредитный рейтинг ПАО «ФосАгро» на уровне <rating> Москва, 30 марта 2023 г. Рейтинговое агентство «Эксперт РА» присвоило рейтинг кредитоспособности нефинансовой компании ПАО «ФосАгро» на уровне <rating>. Прогноз по рейтингу – стабильный. ") Группа «ФосАгро» является одним из ведущих в мире производителей минеральных удобрений на фосфорной основе, а также азотных и сложных удобрений. ПАО «ФосАгро» (далее – компания) является материнской консолидирующей компанией для добывающих, производственных активов и сегмента дистрибуции Группы «ФосАгро» (далее – Группа). Ключевым перерабатывающим активом Группы является АО «Апатит» в г. Череповец совместно с Балаковским и Волховским филиалом. Добыча фосфатного сырья осуществляется в г. Кировск в Мурманской области. Риск-профиль отрасли оказывает поддержку уровню рейтинга. Оценка устойчивости к внешним шокам ограничивается высокой волатильностью цен на минеральные удобрения, при этом исторически Группа демонстрирует сильные оп

In [ ]:
tokens_and_weights = explain(model, X_test[1])
draw_html([(tok, weight * 10) for tok, weight in tokens_and_weights]);

«Эксперт РА» присвоил кредитный рейтинг ПАО «ФосАгро» на уровне Москва, 30 марта 2023 г. Рейтинговое агентство «Эксперт РА» присвоило рейтинг кредитоспособности нефинансовой компании ПАО «ФосАгро» на уровне . Прогноз по рейтингу – стабильный. ") Группа «ФосАгро» является одним из ведущих в мире производителей минеральных удобрений на фосфорной основе, а также азотных и сложных удобрений. ПАО «ФосАгро» (далее – компания) является материнской консолидирующей компанией для добывающих, производственных активов и сегмента дистрибуции Группы «ФосАгро» (далее – Группа). Ключевым перерабатывающим активом Группы является АО «Апатит» в г. Череповец совместно с Балаковским и Волховским филиалом. Добыча фосфатного сырья осуществляется в г. Кировск в Мурманской области. Риск-профиль отрасли оказывает поддержку уровню рейтинга. Оценка устойчивости к внешним шокам ограничивается высокой волатильностью цен на минеральные удобрения, при этом исторически Группа демонстрирует сильные операционные и финансовые результаты, а также поддерживает относительно низкую долговую нагрузку, в том числе и в периоды негативной ценовой конъюнктуры. Показатели рентабельности Группы находятся на уровне выше среднерыночных и сопоставимы с мировыми лидерами отрасли. Агентство отмечает невысокую степень влияния геополитических ограничений на деятельность Группы ввиду обеспечения отраслью минеральных удобрений мировой продовольственной безопасности. Высокая степень географической диверсификации снижает риски падения общих объемов продаж в отдельных регионах. Группе в значительной мере удалось сохранить цепочки поставок в условиях геополитической напряженности, также частично продукция была перенаправлена на другие рынки сбыта, в частности, значительно выросла доля Индии. За отчетный период Группа нарастила объем продаж минеральных удобрений в натуральном выражении на 6,8%, что свидетельствует об устойчивости поставок в условиях макроэкономических шоков. Барьеры для входа в отрасль Группы оцениваются как высокие исходя из уровня капиталоемкости производства. Согласно

In [ ]:
tokens_and_weights

[('«Эксперт', tensor(-0.1911)),
 ('РА»', tensor(-0.1752)),
 ('присвоил', tensor(-0.0146)),
 ('кредитный', tensor(-0.0089)),
 ('рейтинг', tensor(-0.0084)),
 ('ПАО', tensor(0.0531)),
 ('«ФосАгро»', tensor(0.0016)),
 ('на', tensor(-0.0030)),
 ('уровне', tensor(0.0106)),
 ('<rating>', tensor(-0.2350)),
 ('Москва,', tensor(-0.0788)),
 ('30', tensor(0.0028)),
 ('марта', tensor(0.0064)),
 ('2023', tensor(0.0079)),
 ('г.', tensor(-0.0734)),
 ('Рейтинговое', tensor(-0.1027)),
 ('агентство', tensor(-0.0134)),
 ('«Эксперт', tensor(-0.0887)),
 ('РА»', tensor(-0.0835)),
 ('присвоило', tensor(-0.0756)),
 ('рейтинг', tensor(-0.0055)),
 ('кредитоспособности', tensor(-0.2077)),
 ('нефинансовой', tensor(-0.2173)),
 ('компании', tensor(-0.0052)),
 ('ПАО', tensor(0.0143)),
 ('«ФосАгро»', tensor(-0.0588)),
 ('на', tensor(-0.0023)),
 ('уровне', tensor(0.0026)),
 ('<rating>.', tensor(-0.1842)),
 ('Прогноз', tensor(-0.0009)),
 ('по', tensor(-0.0024)),
 ('рейтингу', tensor(-0.0003)),
 ('–', tensor(0.)),
 ('ста

In [ ]:
def convert_from_torch_to_onnx(
        onnx_path: str,
        tokenizer: BertTokenizer,
        model: BertForSequenceClassification
) -> None:
    """Конвертация модели из формата PyTorch в формат ONNX.

    @param onnx_path: путь к модели в формате ONNX
    @param tokenizer: токенизатор
    @param model: модель
    """
    dummy_model_input = tokenizer(
        "текст для конвертации",
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    ).to("cuda")
    torch.onnx.export(
        model,
        dummy_model_input["input_ids"],
        onnx_path,
        opset_version=12,
        input_names=["input_ids"],
        output_names=["output"],
        dynamic_axes={
            "input_ids": {
                0: "batch_size",
                1: "sequence_len"
            },
            "output": {
                0: "batch_size"
            }
        }
    )


In [ ]:
convert_from_torch_to_onnx('rubert-tiny2-cat-7764.onnx', tokenizer, model)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
from onnxruntime.quantization import (
    quantize_dynamic,
    QuantType
)

def convert_from_onnx_to_quantized_onnx(
        onnx_model_path: str,
        quantized_onnx_model_path: str
) -> None:
    """Квантизация модели в формате ONNX до Int8
    и сохранение кванитизированной модели на диск.

    @param onnx_model_path: путь к модели в формате ONNX
    @param quantized_onnx_model_path: путь к квантизированной модели
    """
    quantize_dynamic(
        onnx_model_path,
        quantized_onnx_model_path,
        weight_type=QuantType.QUInt8
    )

In [ ]:
convert_from_onnx_to_quantized_onnx('rubert-tiny2-cat-7764.onnx', 'rubert-tiny2-cat-7764-quant.onnx')

Ignore MatMul due to non constant B: /[/bert/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.2/attention/self/MatMul_1]


In [ ]:
import onnxruntime
from onnxruntime import (
    InferenceSession,
    SessionOptions
)


def create_onnx_session(
        model_path: str,
        provider: str = "CPUExecutionProvider"
) -> InferenceSession:
    """Создание сессии для инференса модели с помощью ONNX Runtime.

    @param model_path: путь к модели в формате ONNX
    @param provider: инференс на ЦП
    @return: ONNX Runtime-сессия
    """
    options = SessionOptions()
    options.graph_optimization_level = \
        onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
    session = InferenceSession(model_path, options, providers=[provider])
    session.disable_fallback()
    return session

In [ ]:
sess = create_onnx_session('rubert-tiny2-cat-7764-quant.onnx')

In [ ]:
def onnx_inference(
        text: str,
        session: InferenceSession,
        tokenizer: BertTokenizer,
        max_length: int
) -> np.ndarray:
    """Инференс модели с помощью ONNX Runtime.

    @param text: входной текст для классификации
    @param session: ONNX Runtime-сессия
    @param tokenizer: токенизатор
    @param max_length: максимальная длина последовательности в токенах
    @return: логиты на выходе из модели
    """
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="np",
    )
    input_feed = {
        "input_ids": inputs["input_ids"].astype(np.int64)
    }
    outputs = session.run(
        output_names=["output"],
        input_feed=input_feed
    )[0]
    return outputs

In [ ]:
onnx_inference(X_test[1], sess, tokenizer, maxSequenceLength)

array([[-0.22926746, -3.3321743 ,  7.558688  , -4.4215283 ,  2.0707202 ,
        -0.39730626,  0.26195222]], dtype=float32)